## Create coordiante files 

In [6]:
%matplotlib inline
import healpy as hp
import numpy as np
import pylab as pl

In [7]:
nside = 512

In [8]:
radius=1.5

In [9]:
## all UL sky map center on the neutrino alert best fit position, expect GB6

source= { 'IC171106A': {'ra':340, 'dec': 7.4},
          'IC191001A': {'ra':314.08,'dec': 12.94},
         'IC200107': {'ra':148.18, 'dec': 35.46},
         'IC201222A': {'ra':206.37, 'dec': 13.44},
         'IC201114A': {'ra':105.25, 'dec': 6.05},
         'GB6J0316+0904': {'ra':49.053058, 'dec':  9.078692},
        }

# print(source['IC171106A']['ra'])

In [10]:
# hp.mollview(hmap)

In [11]:
def create_coord(name,ra,dec,spacing):

    degree = np.pi / 180.
    vec = hp.ang2vec(ra, dec, lonlat=True)
    pixn = hp.query_disc(nside, vec, spacing * degree) #list vec +/- 1.5deg
    i = 1
    ras = []
    decs = []
    num=[]
    for p in pixn:
        rapix, decpix = hp.pix2ang(nside,p,lonlat=True)
        ras.append(rapix)
        decs.append(decpix)
        num.append(i)
    #     print(i, rapix, decpix)
        i +=1 
    np.savetxt('coord_'+name, np.c_[num, ras, decs], delimiter='    ')

In [12]:
for i in [*source]: #list the keys in dic
    create_coord(i,source[i]['ra'],source[i]['dec'],radius)

In [13]:
# pl.figure(figsize=(10,10))

# pl.scatter(ras,decs,s=500,marker="h")

# rasq = np.linspace((ra-2), (ra+2), 40)
# decsq = np.linspace(dec-2, dec+2, 40)
# x, y = np.meshgrid(rasq, decsq)
# pl.scatter(x,y,s=7)



# generate parameter files

In [14]:
def generate_para(name):
    with open('coord_'+name, 'r') as file1:

            lines1=file1.readlines()
            number = np.array([])
            RA = np.array([])
            DEC = np.array([])

            for line1 in lines1:
                number=np.append(number,line1.split()[0]) 
                RA=np.append(RA,line1.split()[1]) 
                DEC=np.append(DEC,line1.split()[2]) 

    with open("ANASUM.template", "r") as fin:
         lines2=fin.readlines()

    for i in range(np.size(number)):
    # for i in range(2):
        with open(name+'_para_'+str(int(float(number[i]))), "w") as fout:
                for line2 in lines2:
                    str1=line2.replace('TIMEMASKFILE', 'timemask_NuToO.dat')
                    str2=str1.replace('ENERGYSPECTRALINDEX', 'ENERGYSPECTRALINDEX 2.5')
                    str3=str2.replace('SKYMAPCENTRE_RADECJ2000_DEG', 'SKYMAPCENTRE_RADECJ2000_DEG'+'  '+RA[i]+'  '+DEC[i])
                    str4=str3.replace('TARGETPOSITION_RADECJ2000_DEG', 'TARGETPOSITION_RADECJ2000_DEG'+'  '+RA[i]+'  '+DEC[i])
                    fout.write(str4)


for i in [*source]:
    generate_para(i)